In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector

# -------------------- MySQL Setup --------------------
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        
        user="root",
        password="redhat",  # change as needed
        database="banking"
    )

# -------------------- New Account Form --------------------
class NewAccountForm:
    def __init__(self, parent, callback=None):
        self.top = tk.Toplevel(parent)
        self.top.title("Create New Account")
        self.callback = callback

        tk.Label(self.top, text="User ID").grid(row=0, column=0, padx=10, pady=5)
        self.id_entry = tk.Entry(self.top)
        self.id_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(self.top, text="Password").grid(row=1, column=0, padx=10, pady=5)
        self.pass_entry = tk.Entry(self.top, show="*")
        self.pass_entry.grid(row=1, column=1, padx=10, pady=5)

        tk.Label(self.top, text="Name").grid(row=2, column=0, padx=10, pady=5)
        self.name_entry = tk.Entry(self.top)
        self.name_entry.grid(row=2, column=1, padx=10, pady=5)

        tk.Label(self.top, text="Initial Balance").grid(row=3, column=0, padx=10, pady=5)
        self.balance_entry = tk.Entry(self.top)
        self.balance_entry.grid(row=3, column=1, padx=10, pady=5)

        tk.Button(self.top, text="Create Account", command=self.submit).grid(row=4, column=0, columnspan=2, pady=10)

    def submit(self):
        try:
            user_id = int(self.id_entry.get())
            password = int(self.pass_entry.get())
            name = self.name_entry.get()
            balance = float(self.balance_entry.get())

            conn = get_connection()
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM accounts WHERE ID=%s", (user_id,))
            if cursor.fetchone():
                messagebox.showerror("Error", "ID already exists!")
                conn.close()
                return

            cursor.execute("INSERT INTO accounts (ID, Password, Name, TotalB) VALUES (%s, %s, %s, %s)",
                           (user_id, password, name, balance))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Account created successfully!")
            self.top.destroy()
            if self.callback:
                self.callback()
        except Exception as e:
            messagebox.showerror("Error", str(e))


# -------------------- USER FUNCTIONALITY --------------------
class UserWindow:
    def __init__(self, root):
        self.root = root
        self.window = tk.Toplevel(root)
        self.window.title("User Portal")
        self.login_screen()

    def login_screen(self):
        tk.Label(self.window, text="User ID").grid(row=0, column=0)
        tk.Label(self.window, text="Password").grid(row=1, column=0)
        self.id_entry = tk.Entry(self.window)
        self.pass_entry = tk.Entry(self.window, show='*')
        self.id_entry.grid(row=0, column=1)
        self.pass_entry.grid(row=1, column=1)
        tk.Button(self.window, text="Login", command=self.authenticate).grid(row=2, column=1)

    def authenticate(self):
        try:
            self.user_id = int(self.id_entry.get())
            password = int(self.pass_entry.get())
            conn = get_connection()
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT * FROM accounts WHERE ID=%s", (self.user_id,))
            row = cursor.fetchone()
            conn.close()

            if row and row["Password"] == password:
                self.main_menu()
            else:
                messagebox.showerror("Error", "Invalid credentials.")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers!")

    def main_menu(self):
        for widget in self.window.winfo_children():
            widget.destroy()

        tk.Button(self.window, text="Check Balance", bg="#2980b9", fg="white", width=20, font=("Arial", 14),
                  command=self.check_balance).pack(pady=5)
        tk.Button(self.window, text="Deposit", bg="#27ae60", fg="white", width=20, font=("Arial", 14),
                  command=self.deposit).pack(pady=5)
        tk.Button(self.window, text="Withdraw", bg="#f39c12", fg="white", width=20, font=("Arial", 14),
                  command=self.withdraw).pack(pady=5)
        tk.Button(self.window, text="Logout", bg="#c0392b", fg="white", width=20, font=("Arial", 14),
                  command=self.window.destroy).pack(pady=5)

    def check_balance(self):
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT TotalB FROM accounts WHERE ID=%s", (self.user_id,))
        balance = cursor.fetchone()[0]
        conn.close()
        messagebox.showinfo("Balance", f"Your Balance: ₹{balance}")

    def deposit(self):
        from tkinter import simpledialog
        amount = simpledialog.askfloat("Deposit", "Enter amount to deposit:")
        if amount and amount > 0:
            conn = get_connection()
            cursor = conn.cursor()
            cursor.execute("UPDATE accounts SET TotalB = TotalB + %s WHERE ID=%s", (amount, self.user_id))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Deposit successful!")

    def withdraw(self):
        from tkinter import simpledialog
        amount = simpledialog.askfloat("Withdraw", "Enter amount to withdraw:")
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT TotalB FROM accounts WHERE ID=%s", (self.user_id,))
        current = cursor.fetchone()[0]

        if amount is None:
            return
        if amount > current:
            messagebox.showerror("Error", "Insufficient Balance")
        else:
            cursor.execute("UPDATE accounts SET TotalB = TotalB - %s WHERE ID=%s", (amount, self.user_id))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Withdrawal successful!")


# -------------------- MANAGER FUNCTIONALITY --------------------
class ManagerWindow:
    def __init__(self, root):
        self.root = root
        self.window = tk.Toplevel(root)
        self.window.title("Manager Portal")
        self.auth_screen()

    def auth_screen(self):
        tk.Label(self.window, text="Enter Manager Password:").pack()
        self.pass_entry = tk.Entry(self.window, show='*')
        self.pass_entry.pack(pady=5)
        tk.Button(self.window, text="Login", command=self.check_pass).pack()

    def check_pass(self):
        if self.pass_entry.get() == "330709":
            self.main_menu()
        else:
            messagebox.showerror("Error", "Wrong Password!")

    def main_menu(self):
        for widget in self.window.winfo_children():
            widget.destroy()
        tk.Label(self.window, text="Manager Menu", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Button(self.window, text="Check All Accounts", bg="#8e44ad", fg="white", width=25, font=("Arial", 14),
                  command=self.show_all).pack(pady=5)
        tk.Button(self.window, text="Add New Account", bg="#27ae60", fg="white", width=25, font=("Arial", 14),
                  command=lambda: NewAccountForm(self.window)).pack(pady=5)
        tk.Button(self.window, text="Delete Account", bg="#e67e22", fg="white", width=25, font=("Arial", 14),
                  command=self.delete_account).pack(pady=5)
        tk.Button(self.window, text="Logout", bg="#c0392b", fg="white", width=25, font=("Arial", 14),
                  command=self.window.destroy).pack(pady=5)

    def show_all(self):
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM accounts")
        rows = cursor.fetchall()
        conn.close()

        if not rows:
            messagebox.showinfo("Info", "No accounts found.")
            return

        display_window = tk.Toplevel(self.window)
        display_window.title("All Accounts")
        display_window.geometry("800x400")

        columns = ("ID", "Password", "Name", "TotalB")
        tree = ttk.Treeview(display_window, columns=columns, show="headings")

        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=180, anchor='center')

        for row in rows:
            tree.insert("", tk.END, values=row)

        scrollbar = ttk.Scrollbar(display_window, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)
        scrollbar.pack(side="right", fill="y")
        tree.pack(expand=True, fill="both", padx=10, pady=10)

    def delete_account(self):
        from tkinter import simpledialog
        try:
            del_id = simpledialog.askinteger("Delete ID", "Enter ID to delete:")
            conn = get_connection()
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM accounts WHERE ID=%s", (del_id,))
            if cursor.fetchone():
                confirm = messagebox.askyesno("Confirm", f"Are you sure you want to delete ID {del_id}?")
                if confirm:
                    cursor.execute("DELETE FROM accounts WHERE ID=%s", (del_id,))
                    conn.commit()
                    messagebox.showinfo("Success", "Deleted successfully!")
                else:
                    messagebox.showinfo("Cancelled", "Deletion cancelled.")
            else:
                messagebox.showerror("Error", "ID not found")
            conn.close()
        except Exception as e:
            messagebox.showerror("Error", str(e))


# -------------------- MAIN APP --------------------
def main():
    root = tk.Tk()
    root.title("HJ Bank")
    root.geometry("1550x800+0+0")
    root.configure(bg="lightblue")

    tk.Label(root, text="WELCOME TO 💰 HJ BANK 💰", font=("Arial", 40, "bold"), fg="darkblue", bg="lightblue").pack(pady=20)
    tk.Label(root, text="Since: 2004\n🌐 www.hjbank.com", font=("Arial", 20), bg="lightblue").pack(pady=5)

    btn_style = {"font": ("Arial", 20), "width": 25, "height": 1, "bd": 4, "relief": "raised"}

    tk.Button(root, text="👤 User Login", bg="#3498db", fg="white", command=lambda: UserWindow(root), **btn_style).pack(pady=5)
    tk.Button(root, text="🆕 New User Registration", bg="#9b59b6", fg="white", command=lambda: NewAccountForm(root), **btn_style).pack(pady=5)
    tk.Button(root, text="🛠️ Manager Login", bg="#2ecc71", fg="white", command=lambda: ManagerWindow(root), **btn_style).pack(pady=5)
    tk.Button(root, text="🚪 Exit", bg="#e74c3c", fg="white", command=root.quit, **btn_style).pack(pady=15)

    root.mainloop()


if __name__ == "__main__":
    main()
